In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data.csv', index_col=0)

In [ ]:
# train_set, test_set = split_train_test(df, 0.2)
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print('Finished train test split!')

In [ ]:
# x_train = train_set.iloc[:, 0:-1]
# y_train = train_set.iloc[:, -1]
# x_test = test_set.iloc[:, 0:-1]
# y_test = test_set.iloc[:, -1]

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_train_norm = pd.DataFrame(scaler.fit_transform(x_train))
x_test_norm = pd.DataFrame(scaler.fit_transform(x_test))
print('Normalized x_train \n', x_train_norm.head())
print('Normalized x_test \n', x_test_norm.head())

In [ ]:
indices = y_train
depth = 199
a = tf.one_hot(indices, depth, on_value = 1.0, off_value = 0.0, axis=1)
with tf.Session() as sess:
    y_train_encoded = sess.run(a)
print('y_train head: \n', y_train_encoded[0:5])

In [ ]:
indices = y_test
depth = 199
b = tf.one_hot(indices, depth, on_value = 1.0, off_value = 0.0, axis=1)
with tf.Session() as sess:
    y_test_encoded = sess.run(b)
print('y_test head: \n', y_test_encoded[0:5])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=x_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
          optimizer=sgd,
          metrics=['accuracy'])
model.fit(x_train, y_train_encoded,
          epochs=20,
          batch_size=128, validation_split=0.2)
score = model.evaluate(x_test, y_test_encoded, batch_size=128)
